![atc_radar](images/radar.jpg "Radar")

# TP2-ARCH-05 : Récepteur ADS-B local

## Objectifs pédagogiques

- comprendre ce qu'est un protocole *on demand* ou *broadcast*
- déduire l'architecture du système de surveillance du traffic aérien
- Construire un récepteur ADS-B avec un Raspberry Pi